# Modelo de Shallow Learning

por: 
     Juan José Molina Ocampo  
     Gloria Maritza Zapata González   
     Osiris Contreras Trillos  
     David Stiveen Tamayo Toro  

## Carga de datos

In [2]:
import os
import zipfile
import shutil
import gdown

# Directorio de destino para guardar los datasets
output_dir = './datasets/'

# Crea el directorio si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Función para descargar y extraer archivos comprimidos
def extract_zip(file_path, output_dir):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

# Función para descargar archivos de Google Drive
def download_from_gdrive(file_id, dest_path):
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, dest_path, quiet=False)

# Descarga y extrae el primer archivo comprimido
download_from_gdrive('1WEUch4LcLRC2Upnfg-hnT4pPymov-SFn', os.path.join(output_dir, 'first_dataset.zip'))
extract_zip(os.path.join(output_dir, 'first_dataset.zip'), output_dir)

# Descarga y extrae el segundo archivo comprimido
download_from_gdrive('17c_1I3rrUovfYVDYxn7zkWtCR8Gz62sc', os.path.join(output_dir, 'second_dataset.zip'))
extract_zip(os.path.join(output_dir, 'second_dataset.zip'), output_dir)

# Descarga y extrae el tercer archivo comprimido
download_from_gdrive('1UmoHB1vGfiQ-F-HcGPCp96-hAahA2_89', os.path.join(output_dir, 'third_dataset.zip'))
extract_zip(os.path.join(output_dir, 'third_dataset.zip'), output_dir)

# Descarga y extrae el cuarto archivo comprimido
download_from_gdrive('1djsKtT1CVEZhWNeIR2MrLWE_3jqWDUfy', os.path.join(output_dir, 'fourth_dataset.zip'))
extract_zip(os.path.join(output_dir, 'fourth_dataset.zip'), output_dir)

# Directorio donde descomprimir los archivos combinados
combined_dir = './combined_dataset/'

# Crear el directorio de combinación si no existe
os.makedirs(combined_dir, exist_ok=True)

# Función para combinar los datasets en un solo directorio
def combine_datasets(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            shutil.copy(os.path.join(root, file), output_dir)

# Combina los datasets en un solo directorio
combine_datasets(output_dir, combined_dir)

print(f"Los datasets han sido combinados en {combined_dir}.")


Downloading...
From: https://drive.google.com/uc?id=1WEUch4LcLRC2Upnfg-hnT4pPymov-SFn
To: c:\Users\osiri\Desktop\datasets\first_dataset.zip
100%|██████████| 16.3M/16.3M [00:00<00:00, 19.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=17c_1I3rrUovfYVDYxn7zkWtCR8Gz62sc
From (redirected): https://drive.google.com/uc?id=17c_1I3rrUovfYVDYxn7zkWtCR8Gz62sc&confirm=t&uuid=e92b3173-d877-47db-a2cf-5ebfc9eeeebe
To: c:\Users\osiri\Desktop\datasets\second_dataset.zip
100%|██████████| 42.9M/42.9M [00:01<00:00, 24.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UmoHB1vGfiQ-F-HcGPCp96-hAahA2_89
To: c:\Users\osiri\Desktop\datasets\third_dataset.zip
100%|██████████| 23.5M/23.5M [00:00<00:00, 26.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1djsKtT1CVEZhWNeIR2MrLWE_3jqWDUfy
From (redirected): https://drive.google.com/uc?id=1djsKtT1CVEZhWNeIR2MrLWE_3jqWDUfy&confirm=t&uuid=c18eaa8e-4fa1-4ceb-8fa2-88418ffb6819
To: c:\Users\osiri\Desktop\datasets\four

Los datasets han sido combinados en ./combined_dataset/.


## Limpieza y transformación

In [3]:
from PIL import Image, UnidentifiedImageError

def remove_corrupted_images(combined_dir):
    removed_images = []
    for filename in os.listdir(combined_dir):
        file_path = os.path.join(combined_dir, filename)
        try:
            img = Image.open(file_path)
            img.verify()  # Verifica si la imagen está dañada
            if img.size[0] == 0 or img.size[1] == 0:
                os.remove(file_path)  # Elimina la imagen vacía
                removed_images.append(filename)
        except (IOError, SyntaxError, UnidentifiedImageError):
            os.remove(file_path)  # Elimina la imagen dañada
            removed_images.append(filename)
        except Exception as e:
            print(f"Error al procesar {filename}: {str(e)}")

    return removed_images

removed_images = remove_corrupted_images(combined_dir)

if removed_images:
    print("Se eliminaron las siguientes imágenes dañadas:")
    for filename in removed_images:
        print(filename)
else:
    print("No se encontraron imágenes dañadas.")


Se eliminaron las siguientes imágenes dañadas:
entero_grano3352.png
first_dataset.zip
fourth_dataset.zip
second_dataset.zip
third_dataset.zip


In [4]:
import pandas as pd

# Crear un DataFrame de etiquetas asumiendo que ya has cargado los labels desde los nombres de archivos
image_files = [f for f in os.listdir(combined_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Crear una lista de diccionarios para almacenar los datos
data = []
for image_file in image_files:
    label = image_file.split('_')[0]  # Extraer la etiqueta del nombre del archivo
    data.append({'filename': image_file, 'label': label})

# Convertir la lista de diccionarios a un DataFrame de pandas
labels_df = pd.DataFrame(data)
labels_csv = '/content/combined_dataset/labels.csv'
labels_df.to_csv(labels_csv, index=False)


In [8]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder

# Tamaño de las imágenes (deben coincidir con las dimensiones usadas en la redimension)
image_size = (100, 100)

# Crear arrays para las imágenes y etiquetas
images = []
labels = []

# Cargar y preprocesar imágenes por lotes
batch_size = 1000
num_batches = len(labels_df) // batch_size + 1

def load_and_preprocess_image(filepath):
    try:
        img = load_img(filepath, target_size=image_size)
        img_array = img_to_array(img)
        img_array /= 255.0  # Normalizar los valores de píxeles a [0, 1]
        return img_array
    except UnidentifiedImageError:
        print(f"No se pudo identificar la imagen en {filepath}")
        return None

for batch in range(num_batches):
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, len(labels_df))
    batch_df = labels_df.iloc[start_idx:end_idx]

    for index, row in batch_df.iterrows():
        file_path = os.path.join(combined_dir, row['filename'])
        img_array = load_and_preprocess_image(file_path)
        if img_array is not None:  # Solo agregar si la imagen se cargó correctamente
            images.append(img_array)
            labels.append(row['label'])

    print(f"Procesado lote {batch + 1} de {num_batches}")

images = np.array(images)
labels = np.array(labels)

# Codificar etiquetas
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


Procesado lote 1 de 7
Procesado lote 2 de 7
Procesado lote 3 de 7
Procesado lote 4 de 7
Procesado lote 5 de 7
Procesado lote 6 de 7
Procesado lote 7 de 7


In [9]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

# Aplanar las imágenes para que puedan ser usadas por el SVM
# Las imágenes tienen forma (100, 100, 3) y las convertimos a (100*100*3,)
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)


# Modelo 5 Máquina de Soporte vectorial

In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Crear un pipeline que incluya el escalado de características y el SVM
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=42))

# Entrenar el modelo SVM
svm_model.fit(X_train_flatten, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear', random_state=42))])

In [11]:
from sklearn.metrics import classification_report, accuracy_score

# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test_flatten)

# Evaluar el rendimiento del modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.6924242424242424
Classification Report:
              precision    recall  f1-score   support

      entero       0.73      0.72      0.72       684
      mancha       0.86      0.84      0.85       171
    quebrado       0.82      0.85      0.84       159
        tiza       0.46      0.48      0.47       306

    accuracy                           0.69      1320
   macro avg       0.72      0.72      0.72      1320
weighted avg       0.70      0.69      0.69      1320



# Comparación de resultados con el mejor modelo de redes neuronales, el modelo 4 (convolucional).


* Precisión Global: La red neuronal convolucional (modelo 4) tiene una precisión global del 78.64%, que es significativamente mayor que la precisión del modelo SVM (69.24%).
* Desempeño por Clase: Para las clases "Entero" y "Quebrado", ambos modelos tienen un desempeño relativamente similar, aunque la red neuronal convolucional muestra una ligera ventaja en el recall y F1-score.
Para la clase "Mancha", la red neuronal convolucional tiene una precisión notablemente mayor (0.98) comparada con el SVM (0.86). Sin embargo, el recall del modelo SVM es superior (0.84 vs. 0.73).
Para la clase "Tiza", ambos modelos tienen un desempeño bajo, pero el modelo SVM muestra una ligera mejora en recall y F1-score comparado con la red neuronal.
* Macro y Weighted Averages: La red neuronal convolucional tiene mejores macro y weighted averages en precisión, recall y F1-score, lo que sugiere que, en promedio, maneja mejor la clasificación de todas las clases en comparación con el modelo SVM.

Mejor modelo: La red neuronal convolucional (modelo 4) demuestra un rendimiento superior en comparación con el modelo SVM (modelo 5), especialmente en términos de precisión global y en el manejo de clases complejas como "Mancha". El mejor desempeño del modelo de red neuronal puede atribuirse a su capacidad de capturar características más complejas y patrones de las imágenes debido a las capas convolucionales y de pooling, que no son posibles de capturar con un modelo SVM lineal.

## Conclusiones

El caso de estudio se centró en la clasificación de imágenes de arroz en cuatro categorías utilizando varios modelos de aprendizaje automático y redes neuronales. Se evaluaron y compararon cinco modelos diferentes para determinar el más efectivo en términos de precisión y rendimiento general.

El estudio concluye que las redes neuronales convolucionales son la mejor opción para la clasificación de imágenes de arroz, proporcionando un rendimiento significativamente mejor en comparación con las redes feedforward y los modelos de aprendizaje automático tradicionales como el SVM. 